In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

import yfinance as yf
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
from pprint import pprint

from datetime import datetime
from tqdm import tqdm

In [2]:
# S&P100 전체 종목 이름, 심볼, 섹터 가져오기
res = requests.get(url = 'https://en.wikipedia.org/wiki/S%26P_100',
                    headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) \
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36'})

df_symbol = pd.read_html(res.text)[2]
df_symbol.to_csv('../data/stock_data.csv', index=False)

df_symbol.head()

,Symbol,Name,Sector
0,AAPL,Apple Inc.,Information Technology
1,ABBV,AbbVie,Health Care
2,ABT,Abbott Laboratories,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe Inc.,Information Technology


In [3]:
# 이벤트 데이터 생성
df_stock = pd.read_csv('../data/df_cond1_tidy.csv')
cond = df_stock['Symbol'].isin(df_symbol.Symbol.tolist())
df_cond1 = df_stock[cond]
df_cond1.head()

df_merged = df_cond1.merge(df_symbol, how='left', on='Symbol')
df_event = df_merged[['Date', 'Symbol', 'Name', 'Sector']]
df_event.to_csv('../data/event_data.csv', index=False)
df_event.head()

,Date,Symbol,Name,Sector
0,2025-04-09,AAPL,Apple Inc.,Information Technology
1,2024-03-15,ADBE,Adobe Inc.,Information Technology
2,2024-06-14,ADBE,Adobe Inc.,Information Technology
3,2024-12-12,ADBE,Adobe Inc.,Information Technology
4,2025-03-13,ADBE,Adobe Inc.,Information Technology


In [4]:
# 테스트를 위한 이벤트 조사
np.random.seed(252)
df_event.sample(2)

,Date,Symbol,Name,Sector
75,2024-10-24,TSLA,"Tesla, Inc.",Consumer Discretionary
61,2024-02-08,PYPL,PayPal,Financials


In [5]:
ticker = yf.Ticker("TSLA")

news_list = ticker.news
target_date = datetime(2024, 10, 24).date()

# pprint(news_list)

len(ticker.get_news(count=100))
### ---> 최근 10개까지만 가능한가?
### ---> 공식 문서에 명시되어 있지 않지만 그래 보임 ㅜㅜ

10

In [6]:
# 다른 접근법
from unittest import result
from GoogleNews import GoogleNews
googlenews = GoogleNews(lang='en')
googlenews.get_news('TESLA')
googlenews.set_time_range(start='2024-10-24', end='2024-10-24')



pd.DataFrame(googlenews.result()).head()
# 역시 최신 기사만 불러온다. 이러면 안됨.

,title,desc,date,datetime,link,img,media,site,reporter
0,Tesla’s Full-Self Driving Software Is A Mess. ...,None,17 hours ago,2025-09-23 20:23:27.902019,https://news.google.com/read/CBMitgFBVV95cUxPW...,https://news.google.com/api/attachments/CC8iK0...,Forbes,None,By Alan Ohnsman
1,Tesla stock jumps to new 2025 closing high as ...,None,22 hours ago,2025-09-23 15:23:27.902230,https://news.google.com/read/CBMirgFBVV95cUxOQ...,https://news.google.com/api/attachments/CC8iL0...,Yahoo Finance,None,By Pras Subramanian
2,Tesla Stock Has Soared 27% This Month And This...,None,8 hours ago,2025-09-24 05:23:27.902405,https://news.google.com/read/CBMinwFBVV95cUxPN...,https://news.google.com/api/attachments/CC8iL0...,Investor's Business Daily,None,By Kit Norton
3,Why Tesla Stock Is Rising Again and Microsoft ...,None,14 hours ago,2025-09-23 23:23:27.902579,https://news.google.com/read/CBMigAFBVV95cUxON...,https://news.google.com/api/attachments/CC8iK0...,Barron's,None,By Jack Denton
4,A Hacker Decodes and Reveals More Information ...,None,Yesterday,2025-09-23 13:23:27.902746,https://news.google.com/read/CBMijwFBVV95cUxQc...,https://news.google.com/api/attachments/CC8iJ0...,MotorTrend,None,By Justin Banner


In [ ]:
# 다른 접근법 Gnews
from gnews import GNews
googlenews = GNews(
    language='en',
    country='US'
)

googlenews.start_date = (2024, 10, 23)
googlenews.end_date = (2024, 10, 24)

# results = pd.DataFrame(googlenews.get_news('TESLA'))
# results

# 질 불러와진다. 이대로 사용하자.

KeyboardInterrupt: 

In [ ]:
df_processed = pd.read_csv('../data/articles/csv/filtered_news_20250924_043743.csv')